# Asignment
------------------------------------------


## Sesion 1



### P T n Calculations

### Motor selection

### Finding the right Ka

### Key calculations
In this section, Ka and the efficiency of the motor is calculated based on some design assumptions.
------------

![Table_3_5b](Table_3_5b.png)

Assuming a medium-duty induction motor (moderate start/stop frequency), continuous full-load operation with occasional mild shocks, a drivetrain susceptible to gear pitting, and 20 hours/day operation, the preliminary estimate for the control/amplifier constant is 𝐾𝐴=1.9. In this context 𝐾𝐴 represents the gain between the control command and stator voltage (i.e. Vs=KA⋅Vcontrol).A higher KA increases available stator voltage and therefore torque for a given controller output, but it also increases stator currents and associated copper and iron losses (losses scale nonlinearly with current). The operating conditions (frequent moderate starts, shocks and gear pitting) will reduce overall mechanical efficiency and increase vibration-related losses and bearing wear, so the effective system efficiency will be lower than nameplate motor efficiency. This estimated 𝐾𝐴 should therefore be validated experimentally and may need to be derated to limit thermal stress and prolong gearbox and bearing life.